In [172]:
import pandas as pd

comments_1 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments1.csv")
comments_2 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments2.csv")
comments_3 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments3.csv")
comments_4 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments4.csv")
comments_5 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments5.csv")



In [173]:
df_video = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\videos.csv")

In [174]:
for i, df in enumerate([comments_1, comments_2, comments_3, comments_4,comments_5], start=1):
    df["dataset_id"] = i

# Combine
df_all_comments = pd.concat([comments_1, comments_2, comments_3, comments_4,comments_5], ignore_index=True)

In [175]:
def compute_soe(video_df, comment_df, w_video_like=1, w_fav=2, w_comment=3, w_comment_like=1):
    # smoothing constant (avoid inflated scores for small videos)
    k = int(video_df["viewCount"].median())

    video_df["publishedAt"] = pd.to_datetime(video_df["publishedAt"], errors="coerce", utc=True)

    # 1. Aggregate comment-level data per video
    comment_stats = comment_df.groupby("videoId").agg(
        comment_likes_sum=("likeCount", "sum"),
        comment_count=("commentId", "count")
    ).reset_index()

    # 2. Merge into video-level df
    merged = video_df.merge(comment_stats, on="videoId", how="left")

    # Fill NA where videos had no comments
    merged["comment_likes_sum"] = merged["comment_likes_sum"].fillna(0)
    merged["comment_count"] = merged["comment_count"].fillna(0)

    # 3. Engagement score
    merged["engagement"] = (
        w_video_like * merged["likeCount"] +
        w_fav * merged["favouriteCount"] +
        w_comment * merged["commentCount"] +
        w_comment_like * merged["comment_likes_sum"]
    )

    # 4. Denominator
    merged["denominator"] = merged["viewCount"] + k

    # 5. Final SoE (video level)
    merged["SoE"] = merged["engagement"] / merged["denominator"]

    # 6. Broadcast SoE to comments (comment-level granularity)
    comment_with_soe = comment_df.merge(
        merged[["videoId", "SoE"]],
        on="videoId",
        how="left"
    )

    return comment_with_soe


In [176]:
result_df = compute_soe(df_video, df_all_comments)

print(result_df.head(20))

               kind  commentId  channelId  videoId  authorId  \
0   youtube#comment    1781382      14492    74288   2032536   
1   youtube#comment     289571      14727    79618   3043229   
2   youtube#comment     569077       3314    51826    917006   
3   youtube#comment    2957962       5008    58298   1853470   
4   youtube#comment     673093      21411     1265   2584166   
5   youtube#comment    1525723      18073    69091   1915402   
6   youtube#comment    3079259       2988    91785    885833   
7   youtube#comment     579871      45729    66477   2775194   
8   youtube#comment    4596873      18670    88816   1040123   
9   youtube#comment     910365      25235    31185   1092265   
10  youtube#comment     779733      36205    72492   3158722   
11  youtube#comment    1204214      33644    86518   1756840   
12  youtube#comment     957058      20162    75828   1548100   
13  youtube#comment    4672340      29550    53751   1064748   
14  youtube#comment    2797490       477

In [177]:
def get_comment_depth(comment_id, parent_map):
    """Compute depth of a single comment given a parent mapping."""
    depth = 0
    parent = parent_map.get(comment_id, None)
    while parent is not None:
        depth += 1
        parent = parent_map.get(parent, None)
    return depth


def compute_depths(df):
    """Add a depth column to the DataFrame."""
    df = df.copy()
    if "commentId" not in df.columns or "parentCommentId" not in df.columns:
        raise KeyError("DataFrame must contain 'commentId' and 'parentCommentId' columns")
    parent_map = df.set_index("commentId")["parentCommentId"].to_dict()
    df["depth"] = df["commentId"].apply(lambda cid: get_comment_depth(cid, parent_map))
    return df

In [178]:
result_compute_depths = compute_depths(df_all_comments)
print(result_compute_depths.head(20))

               kind  commentId  channelId  videoId  authorId  \
0   youtube#comment    1781382      14492    74288   2032536   
1   youtube#comment     289571      14727    79618   3043229   
2   youtube#comment     569077       3314    51826    917006   
3   youtube#comment    2957962       5008    58298   1853470   
4   youtube#comment     673093      21411     1265   2584166   
5   youtube#comment    1525723      18073    69091   1915402   
6   youtube#comment    3079259       2988    91785    885833   
7   youtube#comment     579871      45729    66477   2775194   
8   youtube#comment    4596873      18670    88816   1040123   
9   youtube#comment     910365      25235    31185   1092265   
10  youtube#comment     779733      36205    72492   3158722   
11  youtube#comment    1204214      33644    86518   1756840   
12  youtube#comment     957058      20162    75828   1548100   
13  youtube#comment    4672340      29550    53751   1064748   
14  youtube#comment    2797490       477

In [179]:
import numpy as np
def compute_reply_factor(df, depth_weight=0.5):
    df = df.copy()
    if "depth" not in df.columns:
        df = compute_depths(df)

    # Count number of replies for each comment
    reply_counts = df["parentCommentId"].value_counts().to_dict()
    df["numReplies"] = df["commentId"].map(reply_counts).fillna(0).astype(int)

    # ReplyFactor = depth scaling + log of number of replies
    df["ReplyFactor"] = (1 + depth_weight * df["depth"]) * (1 + np.log1p(df["numReplies"]))
    return df

In [180]:
print(df_all_comments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4725012 entries, 0 to 4725011
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   kind             object 
 1   commentId        int64  
 2   channelId        int64  
 3   videoId          int64  
 4   authorId         int64  
 5   textOriginal     object 
 6   parentCommentId  float64
 7   likeCount        int64  
 8   publishedAt      object 
 9   updatedAt        object 
 10  dataset_id       int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 396.5+ MB
None


In [181]:
result_reply_factor = compute_reply_factor(df_all_comments,depth_weight=0.5)

print(result_reply_factor.head())

              kind  commentId  channelId  videoId  authorId  \
0  youtube#comment    1781382      14492    74288   2032536   
1  youtube#comment     289571      14727    79618   3043229   
2  youtube#comment     569077       3314    51826    917006   
3  youtube#comment    2957962       5008    58298   1853470   
4  youtube#comment     673093      21411     1265   2584166   

                                        textOriginal  parentCommentId  \
0  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would ro...              NaN   
1   Apply mashed potato juice and mixed it with curd        3198066.0   
2                         69 missed calls from mars👽              NaN   
3                                               Baaa              NaN   
4    you look like raven from phenomena raven no cap              NaN   

   likeCount                publishedAt                  updatedAt  \
0          0  2023-08-15 21:48:52+00:00  2023-08-15 21:48:52+00:00   
1          0  2023-10-02 13:08:22+00:00  2

In [182]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_relevance_by_video(video_df, comment_df, text_col="textOriginal",
                                 video_meta_cols=("title","description","tags")):
    """
    Compute relevance (cosine similarity) between each comment and its video's metadata.
    Strategy: process per videoId to avoid building a huge global matrix.
    Returns: comment_df copy with added "relevance" column (float in [0,1]).
    """
    comment_df = comment_df.copy()

    # Ensure expected columns exist
    for c in ("videoId", text_col, "commentId"):
        if c not in comment_df.columns:
            raise KeyError(f"comment_df must contain column '{c}'")
    for c in ("videoId",) + video_meta_cols:
        if c not in video_df.columns:
            raise KeyError(f"video_df must contain column '{c}'")

    # Build video metadata text map
    video_df = video_df.copy()
    video_df["video_meta"] = (
        video_df[video_meta_cols[0]].fillna("").astype(str) + " " +
        video_df[video_meta_cols[1]].fillna("").astype(str) + " " +
        video_df[video_meta_cols[2]].fillna("").astype(str)
    )
    video_meta_map = dict(zip(video_df["videoId"], video_df["video_meta"]))

    # Prepare output container
    relevances = np.zeros(len(comment_df), dtype=float)

    # iterate grouped by videoId
    for vid, group in comment_df.groupby("videoId", sort=False):
        idx = group.index  # indices in original comment_df
        video_text = str(video_meta_map.get(vid, "")).strip()
        # collect comment texts
        comments = group[text_col].fillna("").astype(str).str.strip().tolist()

        # If video metadata or all comments empty -> fallback 0
        if (not video_text) or all(not c for c in comments):
            continue  # already initialized to 0.0

        try:
            # Build TF-IDF on (video_meta + comments)
            texts = comments + [video_text]   # last element is video
            vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
            vectors = vectorizer.fit_transform(texts)  # sparse matrix
            comment_vecs = vectors[:-1]   # shape (n_comments, n_features)
            video_vec = vectors[-1]       # shape (1, n_features)

            # cosine similarity between each comment vector and video vector
            sims = cosine_similarity(comment_vecs, video_vec).reshape(-1)
            sims = np.clip(sims, 0.0, 1.0)
            relevances[idx] = sims

        except ValueError:
            # e.g., empty vocabulary (all stopwords)
            relevances[idx] = 0.0

    comment_df["relevance"] = relevances
    return comment_df


In [183]:
result_relevance = calculate_relevance_by_video(df_video,df_all_comments)
print(result_relevance.head())


              kind  commentId  channelId  videoId  authorId  \
0  youtube#comment    1781382      14492    74288   2032536   
1  youtube#comment     289571      14727    79618   3043229   
2  youtube#comment     569077       3314    51826    917006   
3  youtube#comment    2957962       5008    58298   1853470   
4  youtube#comment     673093      21411     1265   2584166   

                                        textOriginal  parentCommentId  \
0  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would ro...              NaN   
1   Apply mashed potato juice and mixed it with curd        3198066.0   
2                         69 missed calls from mars👽              NaN   
3                                               Baaa              NaN   
4    you look like raven from phenomena raven no cap              NaN   

   likeCount                publishedAt                  updatedAt  \
0          0  2023-08-15 21:48:52+00:00  2023-08-15 21:48:52+00:00   
1          0  2023-10-02 13:08:22+00:00  2

In [184]:
print(result_relevance.describe())

          commentId     channelId       videoId      authorId  \
count  4.725012e+06  4.725012e+06  4.725012e+06  4.725012e+06   
mean   2.362507e+06  2.677102e+04  4.699673e+04  1.820729e+06   
std    1.363995e+06  1.503666e+04  2.593627e+04  1.053868e+06   
min    0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
25%    1.181254e+06  1.449200e+04  2.597800e+04  9.060968e+05   
50%    2.362506e+06  2.542500e+04  4.709600e+04  1.813632e+06   
75%    3.543761e+06  4.061800e+04  6.944500e+04  2.731818e+06   
max    4.725015e+06  5.367700e+04  9.285400e+04  3.659440e+06   

       parentCommentId     likeCount    dataset_id     relevance  
count     5.161350e+05  4.725012e+06  4.725012e+06  4.725012e+06  
mean      2.623663e+06  1.012744e+01  2.883605e+00  2.485385e-02  
std       1.215428e+06  5.444689e+02  1.367501e+00  5.162430e-02  
min       5.161510e+05  0.000000e+00  1.000000e+00  0.000000e+00  
25%       1.569545e+06  0.000000e+00  2.000000e+00  0.000000e+00  
50%       2.

In [185]:
print(result_compute_depths.describe())

          commentId     channelId       videoId      authorId  \
count  4.725012e+06  4.725012e+06  4.725012e+06  4.725012e+06   
mean   2.362507e+06  2.677102e+04  4.699673e+04  1.820729e+06   
std    1.363995e+06  1.503666e+04  2.593627e+04  1.053868e+06   
min    0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
25%    1.181254e+06  1.449200e+04  2.597800e+04  9.060968e+05   
50%    2.362506e+06  2.542500e+04  4.709600e+04  1.813632e+06   
75%    3.543761e+06  4.061800e+04  6.944500e+04  2.731818e+06   
max    4.725015e+06  5.367700e+04  9.285400e+04  3.659440e+06   

       parentCommentId     likeCount    dataset_id         depth  
count     5.161350e+05  4.725012e+06  4.725012e+06  4.725012e+06  
mean      2.623663e+06  1.012744e+01  2.883605e+00  1.109235e+00  
std       1.215428e+06  5.444689e+02  1.367501e+00  3.119334e-01  
min       5.161510e+05  0.000000e+00  1.000000e+00  1.000000e+00  
25%       1.569545e+06  0.000000e+00  2.000000e+00  1.000000e+00  
50%       2.

In [186]:
parent_id_count = result_compute_depths["parentCommentId"].value_counts(dropna = False)
print(parent_id_count)


parentCommentId
NaN          4208877
3479586.0          6
3945948.0          6
3589983.0          6
1020618.0          6
              ...   
2080065.0          1
1211588.0          1
2710703.0          1
2339374.0          1
1672112.0          1
Name: count, Length: 317070, dtype: int64


In [187]:
print(result_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4725012 entries, 0 to 4725011
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   kind             object 
 1   commentId        int64  
 2   channelId        int64  
 3   videoId          int64  
 4   authorId         int64  
 5   textOriginal     object 
 6   parentCommentId  float64
 7   likeCount        int64  
 8   publishedAt      object 
 9   updatedAt        object 
 10  dataset_id       int64  
 11  SoE              float64
dtypes: float64(2), int64(6), object(4)
memory usage: 432.6+ MB
None


In [188]:
def compute_time_norm(video_df, comment_df, tz="UTC"):
    today = pd.Timestamp.now(tz=tz)

    # Ensure datetime types
    video_df = video_df.copy()
    comment_df = comment_df.copy()
    video_df["publishedAt"] = pd.to_datetime(video_df["publishedAt"], errors="coerce", utc=True)
    comment_df["publishedAt"] = pd.to_datetime(comment_df["publishedAt"], errors="coerce", utc=True)

    # Compute days since each comment
    comment_df["days_since_comment"] = (today - comment_df["publishedAt"]).dt.days

    # Normalize comment recency
    max_days_comment = comment_df["days_since_comment"].max()

    if max_days_comment == 0 or pd.isna(max_days_comment):
        comment_df["timeNorm"] = 1.0
    else:
        comment_df["timeNorm"] = 1 - (comment_df["days_since_comment"] / max_days_comment)
        comment_df["timeNorm"] = comment_df["timeNorm"].fillna(1.0)

    return comment_df


In [189]:
result_time_norm = compute_time_norm(df_video,df_all_comments,tz="UTC")
print(result_time_norm.head(20))

               kind  commentId  channelId  videoId  authorId  \
0   youtube#comment    1781382      14492    74288   2032536   
1   youtube#comment     289571      14727    79618   3043229   
2   youtube#comment     569077       3314    51826    917006   
3   youtube#comment    2957962       5008    58298   1853470   
4   youtube#comment     673093      21411     1265   2584166   
5   youtube#comment    1525723      18073    69091   1915402   
6   youtube#comment    3079259       2988    91785    885833   
7   youtube#comment     579871      45729    66477   2775194   
8   youtube#comment    4596873      18670    88816   1040123   
9   youtube#comment     910365      25235    31185   1092265   
10  youtube#comment     779733      36205    72492   3158722   
11  youtube#comment    1204214      33644    86518   1756840   
12  youtube#comment     957058      20162    75828   1548100   
13  youtube#comment    4672340      29550    53751   1064748   
14  youtube#comment    2797490       477

In [190]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold

def train_weights(df, target_col="likeCount", log_transform=True, cv_folds=5):
    required_cols = ["SoE", "ReplyFactor", "relevance"]

    # Ensure required columns exist
    for col in required_cols + [target_col]:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' is missing from dataframe")

    # Fill missing values
    X = df[required_cols].fillna(0)
    y = df[target_col].fillna(0)

    # Optional log-transform
    if log_transform:
        y = np.log1p(y)

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train model
    model = LinearRegression()
    model.fit(X_scaled, y)

    # Cross-validation (R² score)
    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_scaled, y, cv=cv, scoring="r2")

    # Extract weights
    results = pd.DataFrame([
        {"Item": "Model", "Value": model},
        {"Item": "Scaler", "Value": scaler},
        {"Item": "Intercept", "Value": model.intercept_},
        {"Item": "CV_Scores", "Value": cv_scores},
        {"Item": "CV_Mean", "Value": np.mean(cv_scores)},
    ])

    # Append weights row by row
    for feature, weight in zip(required_cols, model.coef_):
        results = pd.concat([results, pd.DataFrame([{"Item": feature, "Value": weight}])],
                            ignore_index=True)

    return results


In [191]:
def merge_all_results(soe_results_df, reply_factor_results_df, relevance_relevance_df,time_norm_df):
    merged = soe_results_df.copy()
    merged = merged.merge(
        reply_factor_results_df[["commentId", "ReplyFactor"]],
        on="commentId",
        how="left"
    )

    merged = merged.merge(
        relevance_relevance_df[["commentId", "relevance"]],
        on="commentId",
        how="left"
    )

    merged = merged.merge(
        time_norm_df[["commentId", "timeNorm"]],
        on="commentId",
        how="left"
    )

    return merged


In [206]:
merged_df = merge_all_results(result_df, result_reply_factor, result_relevance,result_time_norm)
print(merged_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4725012 entries, 0 to 4725011
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   kind             object 
 1   commentId        int64  
 2   channelId        int64  
 3   videoId          int64  
 4   authorId         int64  
 5   textOriginal     object 
 6   parentCommentId  float64
 7   likeCount        int64  
 8   publishedAt      object 
 9   updatedAt        object 
 10  dataset_id       int64  
 11  SoE              float64
 12  ReplyFactor      float64
 13  relevance        float64
 14  timeNorm         float64
dtypes: float64(5), int64(6), object(4)
memory usage: 540.7+ MB
None


In [214]:
results_weight = train_weights(merged_df, target_col="likeCount")
soe_value = results_weight.loc[results_weight["Item"] == "SoE", "Value"].values[0]
print(soe_value)
print(results_weight.columns)


0.008940814968675291
Index(['Item', 'Value'], dtype='object')


In [215]:
def compute_final_score(merged_df, weights):
    """
    Compute final score using learned weights from regression.

    Formula:
        FinalScore = intercept + (a*SoE + b*ReplyFactor + c*Relevance) * (TimeNorm if enabled)

    Parameters:
        merged_df : pd.DataFrame
            Must contain columns: SoE, ReplyFactor, relevance
        weights : dict
            Learned weights, e.g. {"SoE": 0.0089, "ReplyFactor": 0.3491, "relevance": 0.0269}
        intercept : float
            Learned intercept (default=0.0 if ignored)
        use_time_norm : bool
            Whether to multiply by TimeNorm if available in merged_df
    """
    soe_weight_value = weights.loc[results_weight["Item"] == "SoE", "Value"].values[0]
    reply_factor_weight_value = weights.loc[results_weight["Item"] == "ReplyFactor", "Value"].values[0]
    relevance_weight_value = weights.loc[results_weight["Item"] == "relevance", "Value"].values[0]




    score = (
        soe_weight_value * merged_df["SoE"]
        + reply_factor_weight_value * merged_df["ReplyFactor"]
        +relevance_weight_value * merged_df["relevance"]
    )

    score = score * merged_df["timeNorm"]

    merged_df["FinalScore"] = score
    return merged_df

In [220]:
results_final_score = compute_final_score(merged_df, results_weight)
print(results_final_score.describe())

          commentId     channelId       videoId      authorId  \
count  4.725012e+06  4.725012e+06  4.725012e+06  4.725012e+06   
mean   2.362507e+06  2.677102e+04  4.699673e+04  1.820729e+06   
std    1.363995e+06  1.503666e+04  2.593627e+04  1.053868e+06   
min    0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
25%    1.181254e+06  1.449200e+04  2.597800e+04  9.060968e+05   
50%    2.362506e+06  2.542500e+04  4.709600e+04  1.813632e+06   
75%    3.543761e+06  4.061800e+04  6.944500e+04  2.731818e+06   
max    4.725015e+06  5.367700e+04  9.285400e+04  3.659440e+06   

       parentCommentId     likeCount    dataset_id           SoE  \
count     5.161350e+05  4.725012e+06  4.725012e+06  4.471305e+06   
mean      2.623663e+06  1.012744e+01  2.883605e+00  5.110901e-02   
std       1.215428e+06  5.444689e+02  1.367501e+00  6.220686e-02   
min       5.161510e+05  0.000000e+00  1.000000e+00  0.000000e+00   
25%       1.569545e+06  0.000000e+00  2.000000e+00  3.014186e-02   
50%   